In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sp
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, f1_score, accuracy_score, recall_score
from sklearn import metrics
from sklearn.mixture import GaussianMixture
from sklearn.metrics import classification_report, roc_auc_score,precision_score
from sklearn.model_selection import StratifiedKFold 
from scipy.stats import f_oneway, kruskal
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

dataSet = sp.loadmat('breastw.mat')
X = dataSet['X']
y = dataSet['y']

skf = StratifiedKFold(n_splits=25) 

test = False
test2 = False
test3 = False
test4 =False

for train_index, test_index in skf.split(X, y):
   

    X_train, X_test = X[train_index,:], X[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    
    gmm = GaussianMixture(n_components=2,init_params='kmeans',covariance_type='diag')
    gmm.fit(X_train,y_train)
    y_pred_gmm=gmm.predict(X_test)
    if precision_score(y_test,y_pred_gmm)< 0.05:

        y_pred_gmm = np.array([0 if label == 1 else 1 for label in y_pred_gmm])
    
    F1=round(f1_score(y_test,y_pred_gmm),3)


    if test==False :
            data = np.array([[F1]])
            test=True
    else :
            data = np.append(data,np.array([F1]).reshape(1,1),axis=0)
    ########
    
    model_KNN = KNeighborsClassifier(n_neighbors=16, weights='distance', algorithm='auto', metric='braycurtis')
    model_KNN.fit(X_train,y_train)
    y_pred_KNN = model_KNN.predict(X_test)
    F1_knn=round(f1_score(y_test,y_pred_KNN),3)
    
    if test2==False :
            KNN = np.array([[F1_knn]])
            test2=True
    else :
            KNN = np.append(KNN,np.array([F1_knn]).reshape(1,1),axis=0)
    ###########
    model_PMC_1 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(8, 2), random_state=1)
    model_PMC_1.fit(X_train, y_train)
    y_pred_PMC_1= model_PMC_1.predict(X_test)
    F1_Neurones=round(f1_score(y_test, y_pred_PMC_1),3)
    if test3==False :
            Neurones = np.array([[F1_Neurones]])
            test3=True
    else :
            Neurones = np.append(Neurones,np.array([F1_Neurones]).reshape(1,1),axis=0)
   #############
    PerceP = Perceptron(penalty='elasticnet', eta0 = 0.01, alpha = 0.1, max_iter = 50) #random_state=42
    PerceP.fit(X_train, y_train)
    y_pred_PSC = PerceP.predict(X_test)
    F1_Perceptron=round(f1_score(y_test, y_pred_PSC),3)
    if test4==False :
            perceptron = np.array([[F1_Perceptron]])
            test4=True
    else :
            perceptron = np.append(perceptron,np.array([F1_Perceptron]).reshape(1,1),axis=0)
   ##########
    
    df = pd.DataFrame(data,columns=['[{}]'.format("Gaussian")])
    df1 = pd.DataFrame(KNN,columns=['[{}]'.format("KNN")])
    df2 = pd.DataFrame(Neurones,columns=['[{}]'.format("Réseaux neurones")])
    df3 = pd.DataFrame(perceptron,columns=['[{}]'.format("Perceptron")])

    Comparaison = pd.concat([df, df1,df2,df3], axis = 1)

display (Comparaison)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.htm

,[Gaussian],[KNN],[Réseaux neurones],[Perceptron]
0,0.833,0.857,0.800,0.870
1,0.909,0.947,0.947,1.000
2,1.000,1.000,0.900,0.909
3,0.909,1.000,0.900,0.909
4,0.909,0.952,0.947,0.947
5,0.952,0.952,0.900,0.909
6,0.909,0.952,0.900,0.952
7,0.870,0.909,0.909,0.737
8,0.900,0.947,0.947,0.947
9,0.800,0.889,0.889,0.889


In [4]:
import scikit_posthocs as sp
t,p = kruskal(Comparaison['[Gaussian]'],Comparaison['[KNN]'],Comparaison['[Réseaux neurones]'],Comparaison['[Perceptron]'])
print('t statistic: %.3f' % t)
print('p value: %.20f' % p)

sp.posthoc_conover([Comparaison['[Gaussian]'],Comparaison['[KNN]'],Comparaison['[Réseaux neurones]'],Comparaison['[Perceptron]']])

t statistic: 8.415
p value: 0.03817376657504192167


,1,2,3,4
1,1.000000,0.004316,0.170417,0.347623
2,0.004316,1.000000,0.126275,0.050595
3,0.170417,0.126275,1.000000,0.662813
4,0.347623,0.050595,0.662813,1.000000
